In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2

In [ ]:
conn = psycopg2.connect(database="?", user="?", password="?", host="?", port="?")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
tablename = 'public.tiki_product'
query = f'''
    create table if not exists {tablename} (
        id serial primary key,
        category_id integer,
        product_id numeric(10,0),
        product_sku numeric(13,0),
        brand_name varchar,
        product_name varchar,
        product_img varchar,
        product_link varchar,
        regular_price numeric(12,2),
        final_price numeric(12,2),
        voucher_price numeric(12,2),
        tiki_now varchar,
        voucher varchar,
        installment varchar,
        review varchar,
        disc_perc varchar,
        rating varchar,
        batch timestamp default current_timestamp
    );
'''
cur.execute(query)
conn.commit()

In [ ]:
query = f'''
    select
        a.category_id, a.category_name, a.url
    from public.tiki_category as a
    left join public.tiki_category as b
        on a.category_id = b.parent_id
    where b.category_name is null;
'''
cur.execute(query)
cat = cur.fetchall()

In [ ]:
def get_web(url):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [ ]:
def crawl_insert(cat, j, articles, k, cur, conn, tablename):

    try:
        category_id = cat[j][0]
        product_id = "0" if articles[k]['data-seller-product-id'] == [] else articles[k]['data-seller-product-id'].strip()
        product_sku = "0" if articles[k]['product-sku'] == [] else articles[k]['product-sku'].strip()
        brand_name = articles[k]['data-brand'].strip().replace("'", "''")
        product_name = articles[k]['data-title'].strip().replace("'", "''")
        product_img = articles[k].img['src'].strip().replace("'", "''")
        product_link = 'https://tiki.vn' + articles[k].a['href'].replace("'", "''")
        final_price = articles[k].find_all('span', {'class': "final-price"})[0].text.strip().replace("đ", "").replace(".", "").split(' ')[0]
        regular_price = final_price if articles[k].find_all('span', {'class': "price-regular"}) == [] else articles[k].find_all('span', {'class': "price-regular"})[0].text.strip().replace("đ", "").replace(".", "").split(' ')[0]
        voucher_price = [final_price if articles[k].find_all('p', {'class': "price-tag"}) == [] else articles[k].find_all('span', {'class': "price"})[0].find('span').text.replace("đ", "").replace(".", "")][0]
        tiki_now = "N" if articles[k].find_all('div', {'class': "badge-service"}) == [] else "Y"
        voucher = ["?" if articles[k].find_all('p', {'class': "price-tag"}) == [] else articles[k].find_all('span', {'class': "code"})[0].text][0]
        installment = ["?" if articles[k].find_all('span', {'class': "installment-price-v2"}) == [] else articles[k].find_all('span', {'class': "installment-price-v2"})[0].text][0]
        review = [articles[k].find_all('p', {'class': "review"})[0].text.strip('\(\)') if articles[k].find_all('p', {'class': "review"}) != [] else "Chưa có nhận xét"][0]
        disc_perc = "0%" if articles[k].find_all('span', {'class': "sale-tag sale-tag-square"}) == [] else articles[k].find_all('span', {'class': "sale-tag sale-tag-square"})[0].text.strip().split(' ')[0]
        rating = ["?" if articles[k].find_all('span', {'class': "rating-content"}) == [] else articles[k].find_all('span', {'class': "rating-content"})[0].find('span')['style'].split(':')[1]][0]
        
        query = f'''insert into {tablename} (category_id, product_id, product_sku, brand_name, product_name, product_img, product_link, regular_price, final_price, voucher_price, tiki_now, disc_perc, voucher, installment, review, rating)
            values('{category_id}', '{product_id}', '{product_sku}', '{brand_name}', '{product_name}', '{product_img}', '{product_link}', '{regular_price}', '{final_price}', '{voucher_price}', '{tiki_now}', '{disc_perc}', '{voucher}', '{installment}', '{review}', '{rating}'
            );
        '''

        cur.execute(query)
        conn.commit()
    except Exception as err:
        print(err)

In [ ]:
%%time
for j in range(len(cat)):
    try:
        soup = get_web(cat[j][2])
        articles = soup.find_all('div', {'class': "product-item"})
        print('Looping: ' + cat[j][2])
        for k in range(len(articles)):
            crawl_insert(cat, j, articles, k, cur, conn, tablename)

        links = soup.find_all('div', {'class': "list-pager"})
        if links != []:
            try:
                while links[0].find_all('a', {'class': "next"}) != []:  
                    soup = get_web('https://tiki.vn' + links[0].find_all('a', {'class': "next"})[0]['href'])
                    articles = soup.find_all('div', {'class': "product-item"})
                    print('Looping: ', cat[j][1], links[0].find_all('a', {'class': "next"})[0]['href'].split('&')[1], sep=' ')
                    for i in range(len(articles)):
                        crawl_insert(cat, j, articles, i, cur, conn, tablename)
                    links = soup.find_all('div', {'class': "list-pager"})
            except Exception as err:
                print(err)
                continue
    except Exception as err:
        print(err)
        continue
        
print('Crawl done!')